In [ ]:
# Copyright 2020 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# FastPitch: Voice Modification with Pre-defined Pitch Transformations

The [FastPitch](https://arxiv.org/abs/2006.06873) model is based on the [FastSpeech](https://arxiv.org/abs/1905.09263) model. Similarly to [FastSpeech2](https://arxiv.org/abs/2006.04558), which has been developed concurrently, it learns to predict the pitch contour and conditions the generation on such contour.

The simple mechanism of predicting the pitch on grapheme-level (rather than frame-level, as FastSpeech2 does) allows to easily alter the pitch during synthesis. FastPitch can thus change the perceived emotional state of the speaker, or slightly emphasise certain lexical units.

## Requirements

Run the notebook inside the container. By default the container forwards port `8888`.
```
bash scripts/docker/interactive.sh

# inside the container
cd notebooks
jupyter notebook --ip='*' --port=8888
```
Please refer the Requirement section in `README.md` for more details and running outside the container.

In [ ]:
import os
assert os.getcwd().split('/')[-1] == 'notebooks'

## Generate audio samples

Training a FastPitch model from scrath takes 3 to 27 hours depending on the type and number of GPUs, performance numbers can be found in Section "Training performance results" in `README.md`. Therefore, to save the time of running this notebook, we recommend to download the pretrained FastPitch checkpoints on NGC for inference.

You can find FP32 checkpoint at [NGC](https://ngc.nvidia.com/catalog/models/nvidia:fastpitch_pyt_fp32_ckpt_v1/files) , and AMP (Automatic Mixed Precision) checkpoint at [NGC](https://ngc.nvidia.com/catalog/models/nvidia:fastpitch_pyt_amp_ckpt_v1/files).

To synthesize audio, you will need a WaveGlow model, which generates waveforms based on mel-spectrograms generated by FastPitch.You can download a pre-trained WaveGlow AMP model at [NGC](https://ngc.nvidia.com/catalog/models/nvidia:waveglow256pyt_fp16).

In [ ]:
! mkdir -p output
! MODEL_DIR='../pretrained_models' ../scripts/download_fastpitch.sh
! MODEL_DIR='../pretrained_models' ../scripts/download_waveglow.sh

You can perform inference using the respective checkpoints that are passed as `--fastpitch` and `--waveglow` arguments. Next, you will use FastPitch model to generate audio samples for input text, including the basic version and the variations i npace, fade out, and pitch transforms, etc.

In [ ]:
import IPython

# store paths in aux variables
fastp = '../pretrained_models/fastpitch/nvidia_fastpitch_200518.pt'
waveg = '../pretrained_models/waveglow/waveglow_1076430_14000_amp.pt'
flags = f'--cuda --fastpitch {fastp} --waveglow {waveg} --wn-channels 256'

### 1. Basic speech synthesis

You need to create an input file with some text, or just input the text in the below cell:

In [ ]:
%%writefile text.txt
The forms of printed letters should be beautiful, and that their arrangement on the page should be reasonable and a help to the shapeliness of the letters themselves.

Run the script below to generate audio from the input text file:

In [ ]:
# basic systhesis
!python ../inference.py {flags} -i text.txt -o output/original > /dev/null

IPython.display.Audio("output/original/audio_0.wav")

### 2. Add variations to the generated speech

FastPitch allows us to exert additional control over the synthesized utterances, the key parameters are the pace, fade out, and pitch transforms in particular.

### 2.1 Pace

FastPitch allows you to linearly adjust the pace of synthesized speech, similar to [FastSpeech](https://arxiv.org/abs/1905.09263) model. For instance, pass --pace 0.5 for a twofold decrease in speed, --pace 1.0 = unchanged.

In [ ]:
# Change the pace of speech to double with --pace 0.5
# (1.0 = unchanged)
!python ../inference.py {flags} -i text.txt -o output/pace --pace 0.5 > /dev/null

IPython.display.Audio("output/pace/audio_0.wav")

### 2.2 Raise or lower the pitch

For every input character, the model predicts a pitch cue - an average pitch over a character in Hz. Pitch can be adjusted by transforming those pitch cues. A few simple examples are provided below.

In [ ]:
# Raise/lower pitch by --pitch-transform-shift <Hz>
# Synthesize with a -50 Hz shift
!python ../inference.py {flags} -i text.txt -o output/riselowpitch --pitch-transform-shift -50 > /dev/null

IPython.display.Audio("output/riselowpitch/audio_0.wav")

### 2.3 Flatten the pitch

In [ ]:
# Flatten the pitch to a constant value with --pitch-transform-flatten
!python ../inference.py {flags} -i text.txt -o output/flattenpitch --pitch-transform-flatten > /dev/null

IPython.display.Audio("output/flattenpitch/audio_0.wav")

### 2.4 Invert the pitch

In [ ]:
# Invert pitch wrt. to the mean pitch with --pitch-transform-invert
!python ../inference.py {flags} -i text.txt -o output/invertpitch --pitch-transform-invert > /dev/null

IPython.display.Audio("output/invertpitch/audio_0.wav")

### 2.5 Amplify the pitch 

In [ ]:
# Amplify pitch wrt. to the mean pitch with --pitch-transform-amplify 2.0
# values in the (1.0, 3.0) range work the best
!python ../inference.py {flags} -i text.txt -o output/amplifypitch --pitch-transform-amplify 2.0 > /dev/null

IPython.display.Audio("output/amplifypitch/audio_0.wav")

### 2.6 Combine the flags

The flags can be combined. You can find all the available options by calling python inference.py --help.

In [ ]:
!python ../inference.py --help

Below example shows how to generate an audio with a combination of the flags --pace --pitch-transform-flatten --pitch-transform-shift --pitch-transform-invert --pitch-transform-amplify

In [ ]:
# Dobuble the speed and combine multiple transformations
!python ../inference.py {flags} -i text.txt -o output/combine \
    --pace 2.0 --pitch-transform-flatten --pitch-transform-shift 50 \
    --pitch-transform-invert --pitch-transform-amplify 1.5 > /dev/null

IPython.display.Audio("output/combine/audio_0.wav")

### 3. Inference performance benchmark

In [ ]:
# Benchmark inference using AMP
!python ../inference.py {flags} \
    --include-warmup --batch-size 8 --repeats 100 --torchscript --amp \
    -i ../phrases/benchmark_8_128.tsv -o output/benchmark

### 4. Next step

Now you have learnt how to generate high quality audio from text using FastPitch, as well as add variations to the audio using the flags. You can experiment with more input texts, or change the hyperparameters of the models, such as pitch flags, batch size, different precisions, etc, to see if they could improve the inference results.

If you are interested in learning more about FastPitch, please check more samples (trained with multi-speaker) presented at [samples page](https://fastpitch.github.io/).